In [7]:
import requests
import re
from bs4 import BeautifulSoup 

In [84]:
def pridej_rocnik(adresa):
    
    response = requests.get(adresa)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    rocnik = []
    
    letapane = re.search("\d\d\d\d", soup.find(class_="content").text).group(0)
    
    divs = soup.find_all('div', {'data-groupnames': ['0', '1']})

    kategorie = ""
    
    for div in divs:
        
        kategorie = div.text.strip()
        
        next_div = div.find_next('div')
        
        while next_div and 'data-groupnames' not in next_div.attrs:
        
            if next_div.find("a"):

                odkazy = next_div.find_all("a")
                
                for o in odkazy:
                    if 'creator' in o['href']:
    
                        radek = {}
                        
                        radek['rok'] = letapane
                        radek['kategorie'] = kategorie
                        
                        radek['kdo'] = o.text.strip()
                
                        radek['film'] = None
                
                        for d in odkazy:
                            if 'movie=' in d['href']:
                                radek['film'] = d.text.strip()
                                            
                        radek['co'] = 'nominace'
                        if 'DRŽITEL' in next_div.text:
                            radek['co'] = 'vyhra'

                        rocnik.append(radek)

            next_div = next_div.find_next('div')
            
    rocnik_cisty = []
    for dictionary in rocnik:
        if dictionary not in rocnik_cisty:
            rocnik_cisty.append(dictionary)
            
    return rocnik_cisty

In [80]:
pridej_rocnik("https://www.filmovaakademie.cz/cz/2021/nominace")

Nejlepší celovečerní hraný film
Nejlepší režie
Nejlepší dokumentární film
Nejlepší herečka v hlavní roli
Nejlepší herec v hlavní roli
Nejlepší herečka ve vedlejší roli
Nejlepší herec ve vedlejší roli
Nejlepší scénář
Nejlepší kamera
Nejlepší střih
Nejlepší zvuk
Nejlepší hudba
Nejlepší scénografie
Nejlepší kostýmy
Nejlepší masky
Nejlepší televizní film nebo minisérie
Nejlepší televizní seriál
Nejlepší animovaný film
Nejlepší krátký film
Mimořádný přínos české kinematografii
Cena Magnesia za nejlepší studentský film
Nejlepší filmový plakát
Cena filmových fanoušků


[{'rok': '2021',
  'kategorie': 'Nejlepší celovečerní hraný film',
  'kdo': 'Jiří Konečný',
  'film': 'Atlas ptáků',
  'co': 'nominace'},
 {'rok': '2021',
  'kategorie': 'Nejlepší celovečerní hraný film',
  'kdo': 'Zuzana Mistríková',
  'film': 'Chyby',
  'co': 'nominace'},
 {'rok': '2021',
  'kategorie': 'Nejlepší celovečerní hraný film',
  'kdo': 'Ľubica Orechovská',
  'film': 'Chyby',
  'co': 'nominace'},
 {'rok': '2021',
  'kategorie': 'Nejlepší celovečerní hraný film',
  'kdo': 'Jan Prušinovský',
  'film': 'Chyby',
  'co': 'nominace'},
 {'rok': '2021',
  'kategorie': 'Nejlepší celovečerní hraný film',
  'kdo': 'Ondřej Zima',
  'film': 'Chyby',
  'co': 'nominace'},
 {'rok': '2021',
  'kategorie': 'Nejlepší celovečerní hraný film',
  'kdo': 'Rudolf Biermann',
  'film': 'Muž se zaječíma ušima',
  'co': 'nominace'},
 {'rok': '2021',
  'kategorie': 'Nejlepší celovečerní hraný film',
  'kdo': 'Martin Šulík',
  'film': 'Muž se zaječíma ušima',
  'co': 'nominace'},
 {'rok': '2021',
  'kat

In [81]:
roky = range(1993,2025)

In [82]:
import pandas as pd

In [88]:
df = pd.DataFrame()
for r in roky:
    print(r)
    df = pd.concat([df, pd.DataFrame(pridej_rocnik(f"https://www.filmovaakademie.cz/cz/{r}/nominace"))])
df

1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


,rok,kategorie,kdo,film,co
0,1993,Nejlepší film,Jiří Ježek,Šakalí léta,vyhra
1,1993,Nejlepší režie,Jan Hřebejk,Šakalí léta,vyhra
2,1993,Hlavní ženský herecký výkon,Jiřina Bohdalová,Nesmrtelná teta,vyhra
3,1993,Hlavní mužský herecký výkon,Josef Abrhám,Šakalí léta,vyhra
4,1993,Nejlepší scénář,Václav Šašek,Helimadoe,vyhra
...,...,...,...,...,...
137,2023,Cena Magnesia za nejlepší studentský film,Anna Wowra,Jsme v tom spolu,nominace
138,2023,Cena Magnesia za nejlepší studentský film,Petr Pylypčuk,Osmý den,nominace
139,2023,Nejlepší filmový plakát,Zuzana Burgrová,Němá tajemství,vyhra
140,2023,Nejlepší filmový plakát,Robert V. Novák,Němá tajemství,vyhra


In [92]:
df = df.reset_index()

In [90]:
import os

In [93]:
df.to_json(os.path.join('data','ceske_lvy.json'))